In [ ]:
!pip install langchain                # Core
!pip install langchain-community      # Community loaders & tools (PyPDFLoader etc.)
!pip install langchain-qdrant         # Qdrant integration
!pip install langchain-huggingface    # HuggingFace embeddings
!pip install langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 64.2 MB/s eta 0:00:00
  Attempting un

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install qdrant-client



In [ ]:
!pip install sentence-transformers

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_cohere import ChatCohere

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams



In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import pandas as pd

In [ ]:
loader = PyPDFLoader("/content/Dress Code and Uniform Policy for Students.pdf")

In [ ]:
docs = loader.load()


In [ ]:
docs

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-07-20T07:33:45+00:00', 'author': 'user', 'moddate': '2022-07-20T07:33:45+00:00', 'source': '/content/Dress Code and Uniform Policy for Students.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='DRESS CODE & UNIFORM POLICY FOR STUDENTS \n \nPurpose: The purpose of this Dress Code and Uniform Policy is to outline University -level standards in relation to Dress  \nCode / Uniform that must be followed by students of different disciplines on various occasions. It will help all students to \ncontribute to academic ambience and prestige of the University. Lovely Professional University (LPU) wants to prepare its \nstudents to get accustomed to professional appearance starting from their student life in the University. \n \nDress Code & Uniform for Students:  Distinguished guests, visitors frequently visit the University. Therefore, students  \nmust bear in mind that

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(docs)

In [ ]:
model_name="sentence-transformers/all-mpnet-base-v2"
#model_name="BAAI/bge-large-en-v1.5"
#model_name="nomic-ai/nomic-embed-text-v1.5"

encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(model_name=model_name,encode_kwargs=encode_kwargs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
path_name = "../tmp/langchain_qdrant3"
#path_name = "../embeddings/nomic_ai"

collection_name = "my_data1"  #should be renamed if we have to use or run again

In [ ]:

client = QdrantClient(path=path_name)  #should be run only once

In [ ]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [ ]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

In [ ]:
vector_store.add_documents(documents=chunks)

['17fee95701ce4b19b19f05477d8a33a2',
 'c6c226a2483b46099d7c8aca97adf4e7',
 '2450d5f852e34814b10dd525ee276a8b',
 'e4ee49338b48468fb722cb4e0f955880',
 '40440ec989154e768577244baf4bebf2',
 '5a61196dd5a54e5caa8cdc5c6523b104',
 '7b0e32bbaa714e1897956dfa6ef78acf',
 'd95a25ecd7224c12b251e31601541110',
 '85bf9ea2dd9b4b56bb66a88882750c62',
 'd63dc401261a430da1045a31ed04a81d',
 '43323c1be27442b1a60daa174cfe8430',
 '06e726f485714183bfc7b94c9f8fc166',
 '14acc4b4a4db49e8af606ee9e30b6c05']

In [ ]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [ ]:

print(client.get_collections())

collections=[CollectionDescription(name='my_data1')]


In [ ]:
retriever = vector_store.as_retriever(search_type='mmr',
                          search_kwargs={'k':2}
                         )

In [ ]:

user_query = "Shirt"

In [ ]:
retrieval = retriever.get_relevant_documents(user_query)

/tmp/ipython-input-2066677992.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieval = retriever.get_relevant_documents(user_query)


In [ ]:
retriever.invoke(user_query)

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-07-20T07:33:45+00:00', 'author': 'user', 'moddate': '2022-07-20T07:33:45+00:00', 'source': '/content/Dress Code and Uniform Policy for Students.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', '_id': 'e4ee49338b48468fb722cb4e0f955880', '_collection_name': 'my_data1'}, page_content='Collared half/full sleeves shirt or T-shirt \nwith trousers. \nCollared half/full sleeves shirt or T-shirt \nwith trousers, Kurta with leggings/ \njeggings. \n2 While performing \npracticals in \nLabs/Studios \nStudents (both boys and girls) need to wear lab coats while performing practicals  \nin labs/studios for safety purpose or adhere to instructions as given by the \nconcerned HOL/School. \n3 (i) During events like \nguest lectures, \n*workshops, \nseminars, \nconferences and \nconvocation'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate

In [ ]:
llm = ChatCohere(
    model="command-a-03-2025",
    temperature=0.3,
    max_tokens=300,
    cohere_api_key="dRrV13nDT200p0mV7g3hKyBfmgIAf2YWxspzO4cl"
)

In [ ]:
llm.invoke('hey')

AIMessage(content='Hey! How can I help you today?', additional_kwargs={'id': 'fffb89d5-3602-4037-9f1b-17c28d71c128', 'finish_reason': 'COMPLETE', 'content': 'Hey! How can I help you today?', 'token_count': {'input_tokens': 496.0, 'output_tokens': 11.0}}, response_metadata={'id': 'fffb89d5-3602-4037-9f1b-17c28d71c128', 'finish_reason': 'COMPLETE', 'content': 'Hey! How can I help you today?', 'token_count': {'input_tokens': 496.0, 'output_tokens': 11.0}}, id='run--c29b6b12-f3da-4db4-b6e3-d0c4380a2edd-0', usage_metadata={'input_tokens': 496, 'output_tokens': 11, 'total_tokens': 507})

In [ ]:

prompt = ChatPromptTemplate.from_template("""You are an assistant that answers students' questions about university details, rules, and guidelines.
Always use the provided context to answer.
If the answer is found in the context, give a clean, concise, and clear response in 3–5 sentences.
Do not add information outside the context.
If the context does not contain the answer, say:
"I'm sorry, I could not find that information in the university guidelines."



Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
user_query = "Today is my first day of college , suggest me the outfit to university"

In [ ]:




response = retrieval_chain.invoke({"input": user_query})


In [ ]:


print(response["answer"])

For your first day of college, you should wear a collared half or full sleeves shirt or T-shirt paired with trousers. Alternatively, you can opt for a Kurta with leggings or jeggings. Ensure your outfit is neat and professional, as you are representing the university's image. Avoid casual or overly informal attire, and remember to dress appropriately for any specific activities like lab sessions, where a lab coat may be required.
